In [1]:
pip install finance-datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 2.1 MB/s eta 0:00:00


In [32]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import datetime
import time
import random
from concurrent.futures import ThreadPoolExecutor

In [ ]:
from google.colab import files

# 파일 업로드
uploaded = files.upload()
# 파일명 확인 (업로드한 파일명을 딕셔너리로 반환)
file_name = list(uploaded.keys())[0]

# 엑셀 파일 읽기
df = pd.read_excel(file_name)
df

In [ ]:
df['Code']=df['Code'].astype(str).str.zfill(6)

In [27]:
codes=df['Code'].tolist()
start_date = datetime.date(2014, 1, 1)
end_date = datetime.date(2024, 9, 30)

In [ ]:
import time
import random
from concurrent.futures import ThreadPoolExecutor

def fetch_monthly_close(code):
    time.sleep(random.uniform(0.1, 0.3))  # 짧은 랜덤 지연으로 API 과부하 방지
    df1 = fdr.DataReader(code, start_date, end_date)
    monthly_close = df1['Close'].resample('ME').last()  # 월말 종가만 가져오기
    return code, monthly_close

stock_data = {}
with ThreadPoolExecutor(max_workers=10) as executor:  # max_workers를 최적화된 값으로 조정
    results = executor.map(fetch_monthly_close, codes)
    for code, monthly_close in results:  # 결과 수집
        stock_data[code] = monthly_close

# 모든 종목의 월말 종가 데이터를 하나의 DataFrame으로 결합
result = pd.DataFrame(stock_data)
print(result)


In [46]:
result

,014280,370090,111380,145170,418470,037070,102370,353810,140070,090850,...,335870,196700,051780,053060,121890,069330,106240,084440,011080,096870
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-31,4020,NaN,NaN,NaN,NaN,4002,NaN,NaN,NaN,4075,...,NaN,NaN,1010,2359,2116,20100,788,5100,3296,2785
2014-02-28,5300,NaN,NaN,NaN,NaN,4644,NaN,NaN,NaN,6500,...,NaN,NaN,1160,3746,2014,21500,876,2170,3183,2650
2014-03-31,6250,NaN,NaN,NaN,NaN,4683,NaN,NaN,NaN,6250,...,NaN,NaN,1065,3299,2025,20800,854,1800,3979,2335
2014-04-30,8370,NaN,NaN,NaN,NaN,5325,NaN,NaN,NaN,6550,...,NaN,NaN,992,3184,2179,21400,1009,2385,3693,2860
2014-05-31,10460,NaN,NaN,NaN,NaN,6945,NaN,NaN,NaN,5700,...,NaN,NaN,1275,3104,2240,17400,1013,2190,3693,2580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,5050,22300.0,26600.0,37400.0,19480.0,8770,3965.0,4250.0,3410.0,6380,...,1364.0,1532.0,346,1417,1875,1511,1313,1115,782,2865
2024-06-30,4970,20550.0,22750.0,24650.0,17090.0,10560,3485.0,4455.0,3365.0,6030,...,1310.0,1515.0,351,1427,1884,1402,1237,988,630,2855
2024-07-31,5140,16790.0,19960.0,16500.0,16810.0,9170,3705.0,4470.0,3095.0,5600,...,1284.0,1600.0,312,1280,1921,1504,1176,964,650,2790


In [47]:
# 월별 수익률 계산
month_end_ret = result.pct_change()
month_end_ret_real=month_end_ret.iloc[1:,:]

In [48]:
def momentum_strategy(df, M, N):
    results = []
    total_returns = []  # 총 수익률을 저장할 리스트

    for start_month in range(len(df) - M - N + 1):
        # 룩백 기간 동안의 수익률 누적 계산
        lookback_period = df.iloc[start_month : start_month + M]
        cumulative_returns = lookback_period.add(1).prod() - 1

        # 상위 3개 종목 (롱) 및 하위 3개 종목 (숏) 선정
        long_assets = cumulative_returns.nlargest(3).index
        short_assets = cumulative_returns.nsmallest(3).index

        # 보유 기간 동안의 수익률 계산
        holding_period = df.iloc[start_month + M : start_month + M + N]
        long_return = holding_period[long_assets].add(1).prod(axis=0).mean() - 1
        short_return = holding_period[short_assets].add(1).prod(axis=0).mean() - 1

        # 총 수익률 계산 및 리스트에 추가
        total_return = long_return - short_return
        total_returns.append(total_return)

        # 결과 저장
        results.append({
            'Lookback_Date': df.index[start_month + M - 1],
            'Pfo_Date': df.index[start_month + M + N - 1],  # 해당 보유 기간의 마지막 달
            'Long Assets': list(long_assets),
            'Short Assets': list(short_assets),
            'Long Return': long_return,
            'Short Return': short_return,
            'Total Return': total_return,
        })

    # DataFrame으로 변환
    result_df = pd.DataFrame(results)

    # Tranche Return 열 추가
    result_df['Tranche Return'] = pd.Series(total_returns).rolling(window=N, min_periods=1).mean()

    return result_df

In [56]:
# 데이터프레임과 M, N 값 설정 예시
# monthly_returns = pd.DataFrame()  # 실제 데이터프레임 필요
M = 2  # 룩백 기간
N = 1  # 보유 기간
result_df = momentum_strategy(month_end_ret_real, M, N)
result_df

,Lookback_Date,Pfo_Date,Long Assets,Short Assets,Long Return,Short Return,Total Return,Tranche Return
0,2014-03-31,2014-04-30,"[005320, 037760, 038680]","[084440, 052900, 044960]",0.001816,0.150494,-0.148678,-0.148678
1,2014-04-30,2014-05-31,"[033230, 015710, 046390]","[052900, 044820, 016880]",-0.040634,0.082855,-0.123489,-0.123489
2,2014-05-31,2014-06-30,"[046390, 068050, 013520]","[039010, 060310, 076080]",0.019310,0.066676,-0.047366,-0.047366
3,2014-06-30,2014-07-31,"[005320, 036090, 007980]","[005960, 101330, 006920]",0.192604,0.242536,-0.049932,-0.049932
4,2014-07-31,2014-08-31,"[036090, 024940, 063440]","[001260, 052900, 049550]",-0.157060,0.090166,-0.247226,-0.247226
...,...,...,...,...,...,...,...,...
121,2024-04-30,2024-05-31,"[413630, 900340, 024840]","[318020, 009730, 053160]",0.123228,-0.056952,0.180180,0.180180
122,2024-05-31,2024-06-30,"[016670, 024840, 355150]","[318020, 009620, 360350]",-0.061262,-0.107245,0.045983,0.045983
123,2024-06-30,2024-07-31,"[016670, 103840, 006660]","[430690, 413630, 360350]",-0.140624,-0.094745,-0.045879,-0.045879
124,2024-07-31,2024-08-31,"[070300, 331380, 039610]","[145170, 109670, 107600]",-0.191876,-0.049468,-0.142408,-0.142408


In [57]:
def match_names(row):
    long_names = [df.loc[df['Code'] == symbol, '종목명'].values[0] for symbol in row['Long Assets']]
    short_names = [df.loc[df['Code'] == symbol, '종목명'].values[0] for symbol in row['Short Assets']]
    return pd.Series(long_names + short_names, index=['Long 1', 'Long 2', 'Long 3', 'Short 1', 'Short 2', 'Short 3'])

# 이름 매칭 결과를 새로운 데이터프레임으로 생성
matched_df = result_df.apply(match_names, axis=1)
matched_df.index = result_df['Lookback_Date']  # Lookback_Date를 인덱스로 설정

# 결과 출력
matched_df

,Long 1,Long 2,Long 3,Short 1,Short 2,Short 3
Lookback_Date,,,,,,
2014-03-31,국동,쎄니트,에스넷,유비온,KX하이텍,이글벳
2014-04-30,인성정보,코콤,삼화네트웍스,KX하이텍,코스맥스비티아이,웅진
2014-05-31,삼화네트웍스,팬엔터테인먼트,화승코퍼레이션,현대에이치티,3S,웰크론한텍
2014-06-30,국동,위지트,TP,동부건설,모베이스,모헨즈
2014-07-31,위지트,PN풍년,SM Life Design,남광토건,KX하이텍,잉크테크
...,...,...,...,...,...,...
2024-04-30,씨피시스템,윙입푸드,KBI메탈,포인트모바일,이렘,프리엠스
2024-05-31,디모아,KBI메탈,코스텍시스,포인트모바일,삼보산업,코셈
2024-06-30,디모아,우양,삼성공조,한싹,씨피시스템,코셈


In [58]:
# 트렌치 리턴의 개수와 평균을 기준으로 최적의 M, N 조합을 찾는 함수
def find_best_momentum_strategy(df):
    best_count_M, best_count_N = None, None  # Tranche Return >= 0 개수가 가장 많은 M, N 조합
    best_avg_M, best_avg_N = None, None      # Tranche Return 평균이 가장 높은 M, N 조합
    max_positive_count = 0
    max_avg_return = float('-inf')

    for M in range(1, 10):  # M 값을 1부터 9까지
        for N in range(1, 4):  # N 값을 1부터 3까지
            # momentum_strategy 함수 실행
            result_df = momentum_strategy(df, M, N)

            # Tranche Return 값이 0 이상인 개수 계산
            positive_count = (result_df['Tranche Return'] >= 0).sum()

            # Tranche Return의 평균 계산
            avg_return = result_df['Tranche Return'].mean()

            # Tranche Return >= 0인 개수가 가장 많은 경우 갱신
            if positive_count > max_positive_count:
                max_positive_count = positive_count
                best_count_M, best_count_N = M, N

            # Tranche Return의 평균이 가장 높은 경우 갱신
            if avg_return > max_avg_return:
                max_avg_return = avg_return
                best_avg_M, best_avg_N = M, N

    return (best_count_M, best_count_N, max_positive_count), (best_avg_M, best_avg_N, max_avg_return)

# 예제 실행 (monthly_returns 데이터를 사용해야 함)
# M = 1~9, N = 1~3의 조합 중에서 가장 많은 Tranche Return >= 0인 조합 및 평균이 가장 높은 조합 찾기
best_count_result, best_avg_result = find_best_momentum_strategy(month_end_ret_real)

print(f"Tranche Return >= 0인 개수가 가장 많은 조합: M = {best_count_result[0]}, N = {best_count_result[1]}, 개수 = {best_count_result[2]}")
print(f"Tranche Return 평균이 가장 높은 조합: M = {best_avg_result[0]}, N = {best_avg_result[1]}, 평균 = {best_avg_result[2]}")


Tranche Return >= 0인 개수가 가장 많은 조합: M = 2, N = 1, 개수 = 49
Tranche Return 평균이 가장 높은 조합: M = 1, N = 1, 평균 = -0.06333508682046086


In [ ]:
result_df['Tranche Return'].describe()